### 目的

検査対象のメダルが登録したメダル数種類と同種か判定するモデルの作成と予測を行う  
どれか１つと同一物であれば　OK  
どのメダルとも同種では無い　NG  
的なね  

### 方法

・SiameseNetを、特徴量抽出部分と距離計算部分とに分けて実装  
・店舗で使用するメダルの登録フェーズを追加  
・登録フェーズでは登録メダルの特徴量を保存  
・予測の時に、検査対象のメダルの特徴量抽出ならびに、この特徴量と保存されていた登録メダルの特徴量との距離を計算  
・最終的に、登録メダルのどれか１つと同種であると判定されればOKとする  
・具体的な流れは下記参照  

### 課題
・モデルを２つに分割して１つのモデルとして上手く学習させることはできるのか  
・予測時間は早くなるのか  
・予測の時に２つのモデルを使用しているので、１つのモデルとして書き換える必要がある  
(入力の１つは画像、もう１つは画像から取り出した特徴量で、出力は同一 or 非同一　のモデルを作る的な)  
・モデルの定義の時に、すべてのレイヤーで name 指定をするべき。。  

In [ ]:
import sys, os
from pathlib import Path
import json
import time
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt

BASE_DIR = os.path.abspath(Path().resolve())
sys.path.append(os.path.join(BASE_DIR, "../"))
import TemplateMatching as tm
from prefetch_generator import BackgroundGenerator

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras import optimizers
from ToppanUtils import *
from SiameseNet import *

In [ ]:
tf.__version__, keras.__version__

### メダルデータ準備

In [ ]:
### データが読み込めないーーーーーうわぁああああーーーーーっっっっっっ
data_base_path = "/data_server_storage/clients/kagadenshi/2019.06.05 メダル画像データ(検討用)/TAMA"
imageAs_path = data_base_path + "TAMA/"
imageBs_path = data_base_path + "MARU-1/"

imageAs = [
    cv2.imread(imageAs_path + img_name) for img_name in os.listdir(imageAs_path)
    if img_name != "Thumbs.db"
]
imageBs = [
    cv2.imread(imageBs_path + img_name) for img_name in os.listdir(imageBs_path)
    if img_name != "Thumbs.db"
]

labels = ~

### モデル読み込み for 学習

In [ ]:
### 各種パラメータなどの設定
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

SIZE = 128
CHANNELS = 4
SEED = 1
LR = 1e-5
BATCH_SIZE = 128
EPOCHS = 300

In [ ]:
### モデルの読み込み
# これを何回も実行してしまうと、レイヤーの名前が変わって、部分的なモデルの取り出しがうまくできなくなるかも。。
# したがって、モデルの定義の時にすべてのレイヤーで name 引数の指定をするべき。。
# adam = optimizers.Adam(lr=1e-4)
model = get_net(SIZE, CHANNELS) #, adam)

In [ ]:
### モデルの学習と重みの保存
model.train([imgeAs, imgeBs], labels)
model.save("~.h5") # モデルの保存先を各々変更

### メダルの登録

In [ ]:
### モデルの特徴量抽出部分を取り出す
input_layer_name = "A"
output_layer_name = "model_1"

feature_extract_model = Model(
    inputs=model.get_layer(input_layer_name).input, 
    outputs=model.get_layer(output_layer_name).get_output_at(1)
)


### 取り出した特徴量抽出モデルで、登録したいメダルデータの特徴量抽出
feature_map_As = feature_extract_model.predict(image_As)

In [ ]:
### 登録したいメダルデータの読み込み
imageAs = [
    # 画像を読み込む関数のリスト内包表記したかった。。
]


### 登録したいメダルデータの特徴量を保存
np.save("~.npz", feature_map_As) # 適当なんでちゃんと調べて実装しないとヤバイ

### 検査対象のメダルデータの特徴量抽出ならびに、その特徴量と登録したメダルの特徴量との距離を計算

In [ ]:
### モデルの読み込み
model = load_model() # 適当


### モデルの特徴量抽出部分
input_layer_name = "A" # B - model_0だと上手くいかない
output_layer_name = "model_1"

feature_extract_model = Model(
    inputs=model.get_layer(input_layer_name).input, 
    outputs=model.get_layer(output_layer_name).get_output_at(1)
)


### モデルの距離計算部分
distance_model = get_distance_model(size=SIZE) d

# ここで学習した重みを再度読み込まないといけないので、微妙？
layer_name = "model_2"
distance_model.set_weights(model.get_layer(layer_name).get_weights())


### 不要なモデルは削除し、セッショんを綺麗にする。(関数をいろいろ忘れたし、必要ないかも)
del model
config = tf.ほげほげ()
sess = tf.ふーばー(config)
K.clear_session()
K.set_session(sess)

In [ ]:
### 登録したメダルデータの特徴量を読み込む
feature_map_As = np.load("~.npz")　# 適当


### 取り出したモデルたちでメダルデータの距離計算まで
feature_map_Bs = feature_extract_model.predict(image_Bs)
dists = distance_model.predict([feature_map_As, feature_map_Bs])

### 最終的な判定

ここもまじ適当なので、実装する必要

In [ ]:
if np.any(res.flatten())astype("bool"):
    res = "OK"
else:
    res = "NG"